<a href="https://colab.research.google.com/github/alekriley/alekriley.github.io/blob/master/importance_sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary of Basic MonteCarlo Method
We can approximate the function of a random variable using an average of the function computed over all samples.
> $ E[f(x)]\approx\frac{1}{M}\sum_{m=1}^M f(x^{(m)})$ for samples $x^{(1)},x^{(2)},\dots,x^{(M)}$

In the same way we can approximate the variance of $f(x)$.
>  $ Var[f(x)]\approx\frac{1}{M-1}\sum_{m=1}^M [f(x^{(m)})-u_f]^2$ for samples $x^{(1)},x^{(2)},\dots,x^{(M)}$ with $u_f = \hat E[f(x)]$.

# The Idea of Importance Sampling

Consider the expectation of any function of random variable.

> $$E[f(x)]=\int_x f(x)p(x)dx$$ 

where $p(x)$ is the probability density function corresponding to the random variable x. 
That is, we are taking the expectation with respect to $p(x)$. Notice however that we can easily do the following.
> $$E[f(x)]=\int_x q(x)f(x)\frac{p(x)}{q(x)}dx=E_{q(x)}\left[f(x)\frac{p(x)}{q(x)}\right]$$

We can express the expectation of $f(x)$ with respect to a probability distribution $q(x)$. Typically we refer to $\omega(x^{(i)})=\frac{p(x^{(i)})}{q(x^{(i)})}$ as _importance weights_.

We might choose to do this for a few reasons:
> 1. $q(x)$ is easier to sample from.
> 2. We only know $p(x)$ up to a normalizing constant.
> 3. To reduce the variance of a Monte Carlo Estimate.

The primary focus of the rest of the notebook is on (3) ***variance reduction*** and specifically how _Importance Sampling_ helps us achieve this goal.

Consider a theoretical example where 
> $$q(x) = \frac{f(x)p(x)}{\int_{x} f(x')p(x')dx'}$$ 

and $f(x)\geq0$, then it is clear that the expectation of $f(x)\frac{p(x)}{q(x)}$ is constant with respect to $q(x)$ and thus has ***zero variance***.

While it is clearly impossible to construct such a distribution in practice it nonetheless helps to show that by choosing a _good importance distribution_ $q(x)$ we can reduce the variance of our Monte Carlo Estimate.

At this point it helps to think about what properties make a good _importance distribution_.
> 1. It should be easy to sample from.
> 2. Wherever $p(x)$ is non-zero $q(x)$ should be non-zero.
> 3. It should be easy to compute $q(x)$ for all values of $x$.
> 4. The closer it is to being proportional to $|f(x)|$ the better.

A point to watch out for is that the tails of the distribution matter. If $q(x)$ approaches zero significantly faster than $p(x)$ than the importance weights will be very high (in theory tend towards infinity) and the variance of your estimator will most likely increase. 


## Suppose we only have distribution up to a normalizing constant

Calculate unnormalized importance weights $\hat w^{(i)}=\frac{\hat p(x^{(i)})}{q(x^{(i)})}$ and normalize weights to find importance weights $$\omega^{(i)}=\frac{\hat w^{(i)}}{\sum_k \hat p(x^{(k)})}$$

This is possible because we assume that $p(x^{(i)})=\frac{1}{Z}p(x^{(i)})$ then if we have adequate samples $Z =\sum_k \hat p(x^{(k)})$

Let's do a few examples. First import necessary libraries.

### Application: Estimating rare event probabilities

Consider a density function $F(y)=\int_{-\infty}^y f(x)dx$.
We can rewrite the integral as $$F(y)=\int_{-\infty}^\infty [x\leq y]f(x)dx$$ or in other words as an expectation... which has a corresponding Monte Carlo estimate
> $$E[[x\leq y]] \approx \frac{1}{M}\sum_m [x^{(m)}\leq y]$$

where $[x^{(m)}\leq y]$ is the iverson-bracket which has value one if the condition is true and zero otherwise. The conclusion is we can sample N points from our distribution and count all the ones which are less than y. If we knew nothing about probability this is what we would do!

\\
Let's reformulate the problem using _Importance Sampling_. 
> $$E[[x\leq y]]\approx \frac{1}{M}\sum_m [x^{(m)}\leq y]\frac{p(x^{(m)})}{q(x^{(m)})}$$

Obviously we need to decide on our importance distribution. We want our distribution to be close to being proportional to our iverson-bracket. For our example we can use a transformed exponential distribution.
> $$ Z = -(X-y) \longrightarrow X = -Z+y$$ 

where $Z\sim Exp(\lambda)$. Why is the exponential a good choice? For all $x\gt y$ the $[x\leq y]$ will evaluate to zero. So even though $p(x)\neq0$ for $x\gt y$ the term will evaluate to zero. We will by design use **all** of our samples. We can expect a significantly better estimate.

In [0]:
import numpy as np
import scipy.stats as sts
import seaborn as sne
import matplotlib.pyplot as plt
%matplotlib inline
sne.set_style('dark')

In [99]:
n_samples = 5000
y = -3

actual_p = sts.norm.cdf(y)

####estimate using samples from normal distribution####
x = np.random.randn(n_samples)
estimated_p, std_p = np.mean((x<=y)*1), np.var((x<=y)*1)
print("Actual probability of y less than or equal to {}: {}%".format(y,np.round(100*actual_p,3)))
print("Estimated probability of y less than or equal to {}: {}%".format(y,np.round(100*estimated_p,3)))
print("Variance of estimate: {}".format(np.round(100**2*std_p,4)))

Actual probability of y less than or equal to -3: 0.135%
Estimated probability of y less than or equal to -3: 0.18%
Variance of estimate: 17.9676


In [105]:
z = sts.expon.rvs(size=n_samples)
x = -z+y

ws = sts.norm.pdf(x)/sts.expon.pdf(z)
estimated_p, std_p = np.mean(ws),np.var(ws)
print("Actual probability of y less than or equal to {}: {}%".format(y,np.round(100*actual_p,3)))
print("Estimated probability of y less than or equal to {}: {}%".format(y,np.round(100*estimated_p,3)))
print("Variance of estimate: {}".format(np.round(100**2*std_p,3)))


Actual probability of y less than or equal to -3: 0.135%
Estimated probability of y less than or equal to -3: 0.135%
Variance of estimate: 0.018


## General scheme to sample from an intractable distribution using Importance Sampling

Sample $K$ points from the importance distribution. Calculate unnormalized importance weights $\hat w^{(i)}=\frac{\hat p(x^{(i)})}{q(x^{(i)})}$ and normalize weights to find importance weights $$\omega^{(i)}=\frac{\hat w^{(i)}}{\sum_k \hat p(x^{(k)})}$$

Now we can sample N times with replacement from our vector $(x^{(1)},\dots,x^{(K)})$ with probabilities $(\omega^{(i)},\dots,\omega^{(K)})$ to approximate sampling from our intractable distribution. 

While this is a valid method of sampling from an intractable distribution we do not recommend this approach and would instead suggest some other method either the rejection method or MCMC both not covered here.